## HamLib Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies

This cell is used to pull in dependencies from the benchmark framework, the HamLib utility module, and the newly created observables module (in WIP).

In [1]:
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np
from math import sin, cos, pi
import time

import sys
sys.path.insert(1, "WIP_benchmarks")

# Observable Helper Functions
from observables import *

sys.path[1:1] = ["../_common"]

import hamlib_simulation_kernel
from hamlib_simulation_kernel import HamiltonianSimulation, kernel_draw, get_valid_qubits
from hamlib_simulation_kernel import initial_state, create_circuit   # would like to remove these
from hamlib_utils import create_full_filenames, construct_dataset_name
from hamiltonian_simulation_exact import HamiltonianSimulationExact, HamiltonianSimulation_Noiseless

sys.path[1:1] = ["../../_common", "../../_common/qiskit"]

import execute

execute.verbose = False
execute.verbose_time = False


### Load HamLib Hamiltonian

Here we use the HamLib module to select a Hamiltonian from HamLib and initialize its options.  The API for these functions needs improvement.  The code here is taken from the current HamLib benchmark modules.

In [6]:
import hamlib_simulation_benchmark, hamlib_simulation_kernel, hamlib_utils

hamlib_simulation_benchmark.verbose = False
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

hamlib_simulation_kernel.global_U = None
hamlib_simulation_kernel.global_enc = None
hamlib_simulation_kernel.global_ratio = None
hamlib_simulation_kernel.global_rinst = None
hamlib_simulation_kernel.global_h = None
hamlib_simulation_kernel.global_pbc_val = None

# get key infomation about the selected Hamiltonian
# DEVNOTE: Error handling here can be improved by simply returning False or raising exception
def get_hamiltonian_info(hamiltonian_name=None, init_state=None, K=None, t=None):
    try:
        hamlib_simulation_kernel.filename = create_full_filenames(hamiltonian_name)
        hamlib_simulation_kernel.dataset_name_template = construct_dataset_name(hamlib_simulation_kernel.filename)
    except ValueError:
        print(f"ERROR: cannot load HamLib data for Hamiltonian: {hamiltonian_name}")
        return
    
    if hamlib_simulation_kernel.dataset_name_template == "File key not found in data":
        print(f"ERROR: cannot load HamLib data for Hamiltonian: {hamiltonian_name}")
        return
    
    # Set default parameter values for the hamiltonians
    hamlib_simulation_kernel.set_default_parameter_values(hamlib_simulation_kernel.filename)
        
    # assume default init_state if not given
    if init_state == None:
        init_state = "checkerboard"


### Create a Hamiltonian Simulation Circuit


In [7]:
# create the HamLibSimulation kernel, and the associated Hamiltonian operator

# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
hamiltonian_name = 'TFIM'
hamlib_simulation_kernel.global_h = 2
hamlib_simulation_kernel.global_pbc_val = 'pbc'

init_state = "checkerboard"

get_hamiltonian_info(hamiltonian_name=hamiltonian_name, init_state=init_state)

# Parameters of simulation
num_qubits = 8
K = 1
t = 0.05
        
qc, bitstring, ham_op = HamiltonianSimulation(
    num_qubits, 
    K=K, t=t,
    hamiltonian = hamiltonian_name, 
    init_state = init_state,
    method = 1, 
)

# convert SparsePauliOp to list form
ham_terms = ham_op.to_list()
print(ham_terms)

print(qc)


[('ZZIIIIII', (1+0j)), ('ZIIIIIIZ', (1+0j)), ('IZZIIIII', (1+0j)), ('IIIIIIZZ', (1+0j)), ('IIZZIIII', (1+0j)), ('IIIZZIII', (1+0j)), ('IIIIZZII', (1+0j)), ('IIIIIZZI', (1+0j)), ('XIIIIIII', (2+0j)), ('IXIIIIII', (2+0j)), ('IIIIIIIX', (2+0j)), ('IIXIIIII', (2+0j)), ('IIIXIIII', (2+0j)), ('IIIIXIII', (2+0j)), ('IIIIIXII', (2+0j)), ('IIIIIIXI', (2+0j))]
        ┌───────────┐ ░                                                      »
   q_0: ┤ U3(π,0,π) ├─░────────────────────────■─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_1: ──────────────░────────────────────────┼─────────────────────────────»
        ┌───────────┐ ░                        │                             »
   q_2: ┤ U3(π,0,π) ├─░────────────────────────┼─────────────────────────────»
        └───────────┘ ░                        │                             »
   q_3: ──────────────░────────────────────────┼─────────────────────────────»
        ┌──────

### Function to Create Circuits Directly or using Commuing Groups

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [8]:
# Create circuits optionally optimized by use of commuting groups
def create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups=True):

    # Create circuits from the Hamiltonian directly
    if not use_commuting_groups:   
        print("\n******** creating circuits from Hamiltonian:")
        circuits = create_circuits_ham(num_qubits, ham_terms)
    
    # Convert the Hamiltonian to groups and create the circuits from the groups
    else:   
        print("\n******** creating commuting groups for the Hamiltonian and circuits from the groups:")
        groups = group_commuting_terms_2(ham_terms)
        for i, group in enumerate(groups):
            print(f"Group {i+1}:")
            for pauli, coeff in group:
                print(f"  {pauli}: {coeff}")     
        circuits = create_circuits(num_qubits, groups)

    print(f"\n... constructed {len(circuits)} circuits for this Hamiltonian.")
    return circuits



### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [9]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

# Create circuits from the Hamiltonian 
circuits = create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups)

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits, is_commuting=use_commuting_groups)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating circuits from Hamiltonian:
  ... ZZIIIIII, (1+0j)
  ... ZIIIIIIZ, (1+0j)
  ... IZZIIIII, (1+0j)
  ... IIIIIIZZ, (1+0j)
  ... IIZZIIII, (1+0j)
  ... IIIZZIII, (1+0j)
  ... IIIIZZII, (1+0j)
  ... IIIIIZZI, (1+0j)
  ... XIIIIIII, (2+0j)
  ... IXIIIIII, (2+0j)
  ... IIIIIIIX, (2+0j)
  ... IIXIIIII, (2+0j)
  ... IIIXIIII, (2+0j)
  ... IIIIXIII, (2+0j)
  ... IIIIIXII, (2+0j)
  ... IIIIIIXI, (2+0j)

... constructed 16 circuits for this Hamiltonian.

************ Compute energy of the Hamiltonian 3 times

... transpilation time = 0.648
... total execution time = 0.758
Total Energy: (8.29296875+0j)

... transpilation time = 0.031
... total execution time = 0.138
Total Energy: (7.9140625+0j)

... transpilation time = 0.029
... total execution time = 0.137
Total Energy: (8.21875+0j)



### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [10]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# Create circuits from the Hamiltonian 
circuits = create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups)

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits, is_commuting=use_commuting_groups)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating commuting groups for the Hamiltonian and circuits from the groups:
Group 1:
  ZZIIIIII: (1+0j)
  ZIIIIIIZ: (1+0j)
  IZZIIIII: (1+0j)
  IIIIIIZZ: (1+0j)
  IIZZIIII: (1+0j)
  IIIZZIII: (1+0j)
  IIIIZZII: (1+0j)
  IIIIIZZI: (1+0j)
Group 2:
  XIIIIIII: (2+0j)
  IXIIIIII: (2+0j)
  IIIIIIIX: (2+0j)
  IIXIIIII: (2+0j)
  IIIXIIII: (2+0j)
  IIIIXIII: (2+0j)
  IIIIIXII: (2+0j)
  IIIIIIXI: (2+0j)

... constructed 2 circuits for this Hamiltonian.

************ Compute energy of the Hamiltonian 3 times

... transpilation time = 0.024
... total execution time = 0.046
Total Energy: (8.15625+0j)

... transpilation time = 0.028
... total execution time = 0.044
Total Energy: (7.93359375+0j)

... transpilation time = 0.027
... total execution time = 0.05
Total Energy: (8.1953125+0j)

